# How to use MongoDB with LinkML-Store

...

In [1]:
from linkml_store import Client

client = Client()

In [2]:
db = client.attach_database("mongodb://localhost:27017", "test")

In [3]:
collection = db.create_collection("test", recreate_if_exists=True)

In [4]:
COUNTRIES = "../../tests/input/countries/countries.jsonl"

In [5]:
from linkml_store.utils.format_utils import load_objects

objects = load_objects(COUNTRIES)

In [6]:
import pandas as pd
pd.DataFrame(objects)

,name,code,capital,continent,languages
0,United States,US,"Washington, D.C.",North America,[English]
1,Canada,CA,Ottawa,North America,"[English, French]"
2,Mexico,MX,Mexico City,North America,[Spanish]
3,Brazil,BR,Brasília,South America,[Portuguese]
4,Argentina,AR,Buenos Aires,South America,[Spanish]
5,United Kingdom,GB,London,Europe,[English]
6,France,FR,Paris,Europe,[French]
7,Germany,DE,Berlin,Europe,[German]
8,Italy,IT,Rome,Europe,[Italian]
9,Spain,ES,Madrid,Europe,[Spanish]


In [7]:
collection.insert(objects)

In [8]:
qr = collection.find()

In [9]:
qr.rows_dataframe

,name,code,capital,continent,languages
0,United States,US,"Washington, D.C.",North America,[English]
1,Canada,CA,Ottawa,North America,"[English, French]"
2,Mexico,MX,Mexico City,North America,[Spanish]
3,Brazil,BR,Brasília,South America,[Portuguese]
4,Argentina,AR,Buenos Aires,South America,[Spanish]
5,United Kingdom,GB,London,Europe,[English]
6,France,FR,Paris,Europe,[French]
7,Germany,DE,Berlin,Europe,[German]
8,Italy,IT,Rome,Europe,[Italian]
9,Spain,ES,Madrid,Europe,[Spanish]


In [10]:
qr = collection.find({"continent": "Europe"})

In [11]:
qr.rows_dataframe

,name,code,capital,continent,languages
0,United Kingdom,GB,London,Europe,[English]
1,France,FR,Paris,Europe,[French]
2,Germany,DE,Berlin,Europe,[German]
3,Italy,IT,Rome,Europe,[Italian]
4,Spain,ES,Madrid,Europe,[Spanish]


In [12]:
fc = collection.query_facets()

In [13]:
fc["continent"]

[('Asia', 5),
 ('Europe', 5),
 ('Africa', 3),
 ('North America', 3),
 ('Oceania', 2),
 ('South America', 2)]

In [14]:
collection.attach_indexer("llm")

APIConnectionError: Connection error.

In [ ]:
qr = collection.search("countries with a King or Queen")
qr.rows_dataframe

In [ ]:
qr.num_rows

In [ ]:
qr.ranked_rows